In [3]:
import pandas as pd
import requests

baseUrl = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/'
edition = 'MAIN'
version = '2019-07-31'
uri = baseUrl + 'browser/' + edition + '/' + version + '/concepts/'
def getConcept(sctid):
    resp = requests.get(uri + str(sctid))
    return resp.json()

In [6]:
sctid = '703430008'
concept = getConcept(sctid)
concept['fsn']

{'term': 'Arterial cord blood specimen (specimen)', 'lang': 'en'}

In [15]:
from IPython.display import JSON
JSON(concept)

<IPython.core.display.JSON object>

In [16]:
def getFSN(concept):
    return(concept['fsn']['term'])
getFSN(concept)

'Arterial cord blood specimen (specimen)'

In [17]:
for rel in concept['relationships']:
    print(rel['groupId'], getFSN(concept), getFSN(rel['type']), getFSN(rel['target']), rel['characteristicType'])

0 Arterial cord blood specimen (specimen) Is a (attribute) Cord blood specimen (specimen) INFERRED_RELATIONSHIP
1 Arterial cord blood specimen (specimen) Specimen source topography (attribute) Structure of umbilical blood vessel (body structure) INFERRED_RELATIONSHIP
1 Arterial cord blood specimen (specimen) Specimen substance (attribute) Cord blood (substance) INFERRED_RELATIONSHIP
0 Arterial cord blood specimen (specimen) Is a (attribute) Cord blood specimen (specimen) STATED_RELATIONSHIP
1 Arterial cord blood specimen (specimen) Specimen source topography (attribute) Structure of umbilical blood vessel (body structure) STATED_RELATIONSHIP
1 Arterial cord blood specimen (specimen) Specimen substance (attribute) Cord blood (substance) STATED_RELATIONSHIP


In [18]:
df = pd.DataFrame(concept['relationships'])
df[df['active'] == True][['active', 'destinationId', 'typeId', 'type', 'target',
       'groupId', 'modifier', 'characteristicType']]
df['typeName'] = df.apply(lambda row: row['type']['fsn']['term'], axis=1)
df['targetName'] = df.apply(lambda row: row['target']['fsn']['term'], axis=1)
df.drop(columns=['released', 'releasedEffectiveTime', 'relationshipId',
       'type', 'target', 'moduleId', 'sourceId', 'effectiveTime', 'id'], inplace=True, errors='ignore')
df[df['characteristicType'] == 'INFERRED_RELATIONSHIP'][df['active'] == True]

C:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,active,destinationId,typeId,groupId,modifier,characteristicType,typeName,targetName
0,True,122556008,116680003,0,EXISTENTIAL,INFERRED_RELATIONSHIP,Is a (attribute),Cord blood specimen (specimen)
1,True,408728001,118169006,1,EXISTENTIAL,INFERRED_RELATIONSHIP,Specimen source topography (attribute),Structure of umbilical blood vessel (body stru...
2,True,12499000,370133003,1,EXISTENTIAL,INFERRED_RELATIONSHIP,Specimen substance (attribute),Cord blood (substance)
